Levantando contenedores

In [ ]:
docker run <image_container>

# ejemplo:

docker run hello-world

SyntaxError: ignored

Arrancar un contenedor asignándole un nombre

In [ ]:
docker run --name <name_container> <image_container>

#ejemplo:

docker run --name contenedor_test ubuntu

 Arrancar un contenedor con una terminal interactiva. Pasándole una shell para acceder al contenedor.

In [ ]:
docker run -it <image_container> <shell>

#ejemplo:

docker run -it ubuntu bash


Arrancar un contenedor. Mapeando un puerto del host a un puerto del contenedor. puerto_host : puerto_contenedor

In [ ]:
docker run -p <host_port>:<container_port> <image_container>

#ejemplo:

docker run -p 8080:80 nginx


Igual que el ejemplo anterior pero dejándolo en segundo plano.



In [ ]:
docker run -p 8080:80 -d nginx


Arrancar un contenedor. Que tras terminar su periodo de vida. Será eliminado automáticamente.

In [ ]:
docker run --rm <image_container>

#ejemplo

docker run -p 8080:80 -d --rm nginx


Arrancar un contenedor con un volume.

In [ ]:
docker run -v <volume_name>:<mount_point>:<options> <image_container>

'''
Ejemplo:

Volume -> test
Punto de montaje en el contenedor -> /apps
Opciones -> rw (Lectura y escritura)
'''

docker run -v test:/apps:rw nginx


Arrancar un contenedor con un bind mount.

In [ ]:
docker run -v <shared_folder>:<mount_point>:<options> <image_container>

'''
Ejemplo:

Ruta del host a compartir -> /home/application
Punto de montaje en el contenedor -> /apps
Opciones -> ro (Solo lectura)
'''

docker run -v /home/application:/apps:ro ubuntu


Arrancar un contenedor con tmpfs.

In [ ]:
docker run \
--mount type=tmpfs,destination=<mount_point>,tmpfs-mode=<permisos>,tmpfs-size=<bytes_size> \
<image_container>


'''
Ejemplo:

Punto de montaje en el contenedor -> /temporal
Permisos -> Todos los permisos solo para el propietario.
Tamaño del FS -> 21474836480 bytes = 20G
'''

docker run \
--mount type=tmpfs,destination=/temporal,tmpfs-mode=700,tmpfs-size=21474836480 \
nginx

## Listando contenedores

 Lista de los contenedores activos.

In [ ]:
docker ps

Lista de todos los contenedores activos e inactivos del sistema.

In [ ]:
docker ps -a


Lista los ID de todos los contenedores.

In [ ]:
docker ps -aq


## Debugging

Inspeccionar la data de un contenedor.

In [ ]:
# por su ID:

docker inspect <id_container>


# Por su nombre:

docker inspect <name_container>

# Aplicando filtros. Por ejemplo buscando las variables de entorno:

docker inspect -f '{{ json .Config.Env }}' <name_container>


# Ver los logs del contenedor:

docker logs <name_container>




## Eliminando contenedores

Eliminar un contenedor que no este arriba.

In [ ]:
#  Se puede hacer tanto por nombre como por ID.

docker rm <name_container>




Eliminar un contenedor aunque este arriba. Forzándolo.

In [ ]:
# Se puede hacer tanto por nombre como por ID.

docker rm -f <id_container>


Eliminar todos los contenedores que no esten arriba a la vez.

In [ ]:
docker rm $(docker ps -aq)


## Exponiendo contenedores

In [ ]:
# corro un contenedor
docker run -d --name proxy nginx


# apaga un contenedor
docker stop proxy


# borro el contenedor
docker rm proxy


# lo para y lo borra
docker rm -f proxy


# corro un nginx y expongo el puerto 80 del contenedor en el puerto 8080 de mi máquina
docker run -d --name proxy -p 8080:80 nginx ()


# veo los logs
docker logs proxy


# hago un follow del log
docker logs -f proxy


# veo y sigo las últimas 10 entradas del log
docker logs --tail 10 -f proxy

## Bind mounts

In [ ]:
# Los contenedores son autocontenidos y no pueden acceder a la información de la máquina que los hostea.
#A veces tenemos aplicaciones que generan datos y que queremos acceder a ellos. Crearemos una carpeta nueva para que interactúe con nuestro contenedor

 mkdir dockerdata

#Vamos a crear una base de datos en mongo db y hacer que los datos no mueran con el contenedor cuando se cierre

 docker run -d --name db mongo

# Ahora entraremos al contenedor para generar datos en él:

 docker exec -it db bash

# Ahora estoy dentro del contenedor y accederé a mongo:

 mongo

# ya estoy conectado en la base de datos local y crearé una base de dato llamada "Platzi"

use platzi

# crearé un nuevo usuario llamado "guido"

db.users.insert({“nombre”:“guido”})

# verifico que esté creado:

db.users.find()

# Saldré de mongo y del contenedor con el comando exit dos veces
# y eliminaré el contenedor por completo con docker rm -f db
# Pero al volver de acceder no estará disponible, dado que eliminé el contenedor.
# Para que no se elimine el dato, necesito generar un directorio espejo en mi máquina local, para esto, utilizaremos bind mounts.

# Lo primero que necesitamos es crear un nuevo directorio

mkdir mongodata

# y obtendremos el path en el que está montado:

pwd

# Y copiaremos en el portapapeles la ruta del directorio:

docker run -d --name db -v /miruta/mongodata:/data/db mongo

# donde /miruta/mongodata es la ruta a mi carpeta local y /data/db es la carpeta donde mongo guarda sus datos

# ahora, si repetimos todo el procedimiento anterior, todo quedará espejado en el directorio local, por lo tanto,
# el dato que ingresamos en la base de datos están disponibles en mongo si volvemos a

## Insertar y extraer archivos de un contenedor

> Añadir blockquote



In [ ]:
touch prueba.txt # (creo un archivo en mi máquina)

docker run -d --name copytest ubuntu tail -f /dev/null # (corro un ubuntu y le agrego el tail para que quede activo)

docker exec -it copytest bash # (entro al contenedor)

mkdir testing # (creo un directorio en el contenedor)

docker cp prueba.txt copytest:/testing/test.txt # (copio el archivo dentro del contenedor)

docker cp copytest:/testing localtesting # (copio el directorio de un contenedor a mi máquina) con "docker cp" no hace falta que el contenedor esté corriendo

## Conceptos fundamentales de Docker: imágenes. Son moldes o plantillas que tienen empaquetado todo lo necesario para funcionar. Docker usa estás misma para construir contenedores. ++¿Para qué sirven las imágenes en la construcción de software?++ Las imágenes sirven para crear contenedores y también es como docker intenta solucionar el problema de construcción de y distribución de software.

$ sudo docker image ls Listar imágenes locales de docker. Esta instrucción de terminal mostrará las siguientes columnas

TAG: Es la version de la imagen, cuando no especificamos que versión de descargar docker asume que es la última
CREATE AT: tiempo de creación
SIZE: Tamaño de la imagen
IMAGE ID: Este es el identificador de la imagen. Si descargamos dos imágenes que son iguales al realizar un pull, docker lo que hará es tener un puntero lógico a la misma imagen ya descargada y no descargará la otra imagen. Existe la posibilidad que tenga TAG diferentes, pero serán la misma imagen. Esto último se puede comprobar viendo el IMAGE ID, debería ser los mismos.
$ sudo docker pull Si no colocamos un parámetro después de pull entenderá que deberá realizar la descarga de docker hub. EL parámetro extra después de pull es para indicarle el repositorio

In [ ]:
# lista todas las imágenes

docker image ls

# descargar imágenes desde el Docker Hub

docker pull <image_name>:<tag_version>

Construyendo una imagen propia:

In [ ]:
mkdir imagenes #(creo un directorio en mi máquina)

cd imagenes # (entro al directorio)

touch Dockerfile # (creo un Dockerfile)

code . # (abro code en el direcotrio en el que estoy)

##Contenido del Dockerfile##

FROM ubuntu:latest

RUN touch /ust/src/hola-platzi.txt # comando a ejecutar en tiempo de build

docker build -t ubuntu:platzi . # creo una imagen con el contexto de build <directorio>

docker run -it ubuntu:platzi # corro el contenedor con la nueva imagen

docker login # me logueo en docker hub

docker tag ubuntu:platzi miusuario/ubuntu:platzy # cambio el tag para poder subirla a mi docker hub

docker push miusuario/ubuntu:platzi # publico la imagen a mi docker hub

## Usando Docker para desarrollar Ejemplo como utilizar docker para el desarrollo


Creamos una carpeta y nos posicionamos dentro de esa carpeta.
Luego Clonamos un proyecto de github y nos posicionamos dentro del proyecto clonado
Verificamos que los archivos estén allí
Realizamos la construcción de la imagen,
Parámetro -t o tag : Indica el tag de la imagen a construir. Parámetro . : indica el directorio de trabajo

Verificar que la imagen ha sido creada.
Crear el contenedor basado en la imagen anteriormente creada y:
Parámetro --rm para que, al detenerse. el contenedor se borre Parámetro -p indicar que el puerto del host:contenedor están asociados.

Comprobar que el contenedor recién creado este activo
Comprobar que el contenedor puede recibir petición desde el navegador, mostrará un error porque la nuestro contenedor no está asociado a ninguna bases de mong.

In [ ]:
# Ejecución de los pasos

mkdir docker-pj && cd docker-pj

git clone https://github.com/platzi/docker && cd docker

ll

code .

sudo docker build -t platziapp .

sudo docker image ls

sudo docker run --rm -p 3000:3000 platziapp

sudo docker ps

localhost:3000

In [ ]:
# explicación del Dockerfile

FROM node:12  # Indicamos en que está basado nuestra imagen

COPY [".", "/usr/src/"]  # Indicamos que copie nuestro proyecto en src

WORKDIR /usr/src    # Especificamos un directorio de trabajo, dentro del build

RUN npm install   # Luego ya parado en ese directorio de trabajo ejecutamos npm install, para instalar las dependencias que están en el package.json

EXPOSE 3000   # Se expone el puerto 3000, esto es para poder atar el puerto de la maquina hosts al contenedor

CMD ["node", "index.js"]    # Definimos el proceso o comando por defecto que va a ejecutarse cuando se ejecute el contenedor, command form o exec form
